In [1]:
# Copyright (c) Microsoft Corporation. All rights reserved
# Licensed under the MIT License.
import fiona
import shapely.geometry
import pandas as pd
import subprocess

In [2]:
!wget https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_county_5m.zip
!unzip cb_2018_us_county_5m.zip

--2022-05-11 19:16:34--  https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_county_5m.zip
Resolving www2.census.gov (www2.census.gov)... 23.1.205.135, 2600:1400:11:18b::208c, 2600:1400:11:199::208c
Connecting to www2.census.gov (www2.census.gov)|23.1.205.135|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘cb_2018_us_county_5m.zip’

cb_2018_us_county_5     [ <=>                ]   2.65M  --.-KB/s    in 0.08s   

2022-05-11 19:16:35 (33.9 MB/s) - ‘cb_2018_us_county_5m.zip’ saved [2781997]

Archive:  cb_2018_us_county_5m.zip
  inflating: cb_2018_us_county_5m.shp.ea.iso.xml  
  inflating: cb_2018_us_county_5m.shp.iso.xml  
  inflating: cb_2018_us_county_5m.shp  
  inflating: cb_2018_us_county_5m.shx  
  inflating: cb_2018_us_county_5m.dbf  
  inflating: cb_2018_us_county_5m.prj  
 extracting: cb_2018_us_county_5m.cpg  


In [3]:
# Read the list of county GEOIDs that make up the Chesapeake Bay
geoids = set(pd.read_csv("../data/chesapeake-bay-county-geoids.csv", dtype={"geoids":"str"})["geoids"].values)

In [4]:
# Copy the US counties that are in the Chesapeake Bay into a new GeoJSON file
with fiona.open("cb_2018_us_county_5m.shp") as src:
    dst_schema = src.schema.copy()
    dst_schema["geometry"] = "MultiPolygon"
    
    with fiona.open(
        "../data/chesapeake-bay-counties_epsg4269.geojson",
        mode="w",
        driver="GeoJSON",
        crs=src.crs,
        schema=dst_schema
    ) as dst:
        for row in src:
            if row["properties"]["GEOID"] in geoids:
                if row["geometry"]["type"] == "Polygon":
                    shape = shapely.geometry.shape(row["geometry"])
                    geom = shapely.geometry.mapping(
                        shapely.geometry.MultiPolygon(polygons=[shape])
                    )
                    row["geometry"] = geom
                    dst.write(row)
                else:
                    dst.write(row)

In [5]:
# Convert GeoJSON file to EPSG:4326
assert subprocess.call([
    "ogr2ogr",
    "-t_srs", "epsg:4326",
    "../data/chesapeake-bay-counties_epsg4326.geojson",
    "../data/chesapeake-bay-counties_epsg4269.geojson"
]) == 0

In [6]:
!rm cb_2018_us_county_5m.*